# About

These is a base solution of PID.

In [2]:
%matplotlib inline
import pandas
import numpy
import matplotlib.pyplot as plt
import root_numpy

Couldn't import dot_parser, loading of dot files will not be possible.


# Get files https

In [3]:
def get_files_https(files):

    files_http = []

    for one_file in files:

        one_file_path, one_file_tree_name = one_file.split(':')

        one_file_http = one_file_path.replace("/r02/lhcb/jonesc/ANNPID/", "http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/")

        files_http.append((one_file_http, one_file_tree_name))

    return files_http

In [4]:
path_files_txt = '../../../data/configs/training/MC2015Sim09Dev03-TrainingFiles-Mixture-Cambridge.txt'
files = numpy.loadtxt(path_files_txt, dtype='S', delimiter='\n', comments='#')

files_http = get_files_https(files)

print len(files_http)
files_http[:5]

552


[('http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/ProtoParticlePIDtuples/MC/Sim09Devx/Reco15x/Dev03/ANNPID.1.root',
  'ANNPID/DecayTree'),
 ('http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/ProtoParticlePIDtuples/MC/Sim09Devx/Reco15x/Dev03/ANNPID.2.root',
  'ANNPID/DecayTree'),
 ('http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/ProtoParticlePIDtuples/MC/Sim09Devx/Reco15x/Dev03/ANNPID.3.root',
  'ANNPID/DecayTree'),
 ('http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/ProtoParticlePIDtuples/MC/Sim09Devx/Reco15x/Dev03/ANNPID.4.root',
  'ANNPID/DecayTree'),
 ('http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/ProtoParticlePIDtuples/MC/Sim09Devx/Reco15x/Dev03/ANNPID.5.root',
  'ANNPID/DecayTree')]

# Get numpers of particles

In [5]:
def get_number_particles(files_http, particles_pdg, selection, log_file_name='get_number_particles.log'):
    
    numbers_particles = numpy.zeros((len(files_http), len(particles_pdg)))
    
    LOG = open(log_file_name, 'w')
    LOG.write('Particles pdgs: ' + str(particles_pdg) + '\n')
    LOG.write('Selection: ' + selection + '\n')
    LOG.flush()

    for num, (one_file_https, one_file_tree) in enumerate(files_http):
        
        success = 0    
        while success != 1:
            try:
                mc_particle_type = root_numpy.root2array(filenames=one_file_https, 
                                                         treename=one_file_tree, 
                                                         branches='MCParticleType',
                                                         selection=selection)

                for num_pdg, one_pdg in enumerate(particles_pdg):

                    numbers_particles[num, num_pdg] = (numpy.abs(mc_particle_type) == one_pdg).sum()

                LOG.write(str(num) + '. ' + one_file_https + '\n')
                LOG.write('Numbers of particles: ' + str(numbers_particles[num, :]) + '\n')
                LOG.flush()
                success = 1

            except:
                LOG.write(one_file_https + ' is not readed.' + '\n')
                LOG.flush()

    numbers_particles_df = pandas.DataFrame(data=numbers_particles, columns=[str(i) for i in particles_pdg])
    numbers_particles_df['http'] = numpy.array(files_http)[:, 0]
    numbers_particles_df['tree_name'] = numpy.array(files_http)[:, 1]
    
    return numbers_particles_df

In [ ]:
selection = '(!HasMC || MCVertexType==1 || MCVertexType==2) && \
             (TrackLikelihood > -100.0) && \
             (TrackP > 0) && \
             (TrackPt > 0)'

dir_path = '../../../data/MC2015Sim09Dev03/Generation1'
particles_pdg = [11,13,211,321,2212,0]

In [ ]:
log_file_name = dir_path + '/get_number_particles.log'
numbers_particles_df, no_readed_files = get_number_particles(files_http, 
                                                             particles_pdg, 
                                                             selection, 
                                                             log_file_name=log_file_name)

numbers_particles_df.head()

In [5]:
numbers_particles_df = pandas.read_csv(dir_path + '/number_particles.csv', index_col=0)
#numbers_particles_df.to_csv(dir_path + '/number_particles.csv')
numbers_particles_df.head()

,11,13,211,321,2212,0,http,tree_name
0,2349,27608,400802,58142,46742,162190,http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/...,ANNPID/DecayTree
1,3742,6361,442151,91543,45948,187565,http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/...,ANNPID/DecayTree
2,22990,35439,2227761,329622,241543,954511,http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/...,ANNPID/DecayTree
3,17845,145515,2098478,354946,229897,904141,http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/...,ANNPID/DecayTree
4,18436,152502,2102275,341318,221395,886444,http://www.hep.phy.cam.ac.uk/~jonesc/lhcb/PID/...,ANNPID/DecayTree


# Train generation

In [33]:
from sklearn.cross_validation import train_test_split

def get_sample(numbers_particles, n_tracks, selection, file_path, log_path):
    
    particles = numbers_particles.columns.drop(['http', 'tree_name'])
    part = 1. * n_tracks / numbers_particles[particles].sum()
    
    LOG = open(log_path, 'w')
    LOG.write('Particles pdgs: ' + str(particles) + '\n')
    LOG.write('Number of tracks: ' + str(n_tracks) + '\n')
    LOG.flush()
    
    numbers_per_particle = {}
    for i in particles:
        numbers_per_particle[i] = 0
    
    for index in numbers_particles.index:
        
        success = 0    
        while success != 1:
            try:
                file_http = numbers_particles.loc[index]['http']
                tree_name = numbers_particles.loc[index]['tree_name']

                branches = root_numpy.list_branches(file_http, treename=tree_name)
                branches = numpy.array(branches)

                data_array = root_numpy.root2array(filenames=file_http, 
                                                   treename=tree_name, 
                                                   branches=branches[branches != 'piplus_OWNPV_COV_'],
                                                   selection=selection)

                data = pandas.DataFrame(data=data_array, columns=branches[branches != 'piplus_OWNPV_COV_'])

                LOG.write(file_http + '\n')
                LOG.flush()

                for one_particle in particles:

                    p_type = numpy.abs(data['MCParticleType'].values)
                    data_particle = data[p_type == int(one_particle)]

                    number = numbers_particles.loc[index][one_particle]
                    number_take = int(round(part[one_particle] * number))

                    data_particle_take, _ = train_test_split(data_particle, train_size=number_take, random_state=42)

                    if os.path.exists(file_path):
                        data_particle_take.to_csv(file_path, mode='a', header=False)
                    else:
                        data_particle_take.to_csv(file_path, mode='a', header=True)

                    numbers_per_particle[one_particle] += number_take

                    LOG.write('Particle ' + str(one_particle) + ' : ' + str(number_take) + ' tracks was taken.\n')
                    LOG.flush()

                LOG.write('Tracks selected: ' + str(numbers_per_particle) + '\n')
                LOG.flush()

                success = 1

            except:
                pass
                
    return 1

In [ ]:
index_sel = numpy.array(numbers_particles_df.index) % 2 == 0
numbers_particles_train = numbers_particles_df[index_sel]

get_sample(numbers_particles_train, 1000000, selection, dir_path + '/data_train.csv', dir_path + '/get_sample_train.log')

In [ ]:
data_train = pandas.read_csv(dir_path + '/data_train.csv', index_col=0, usecols=['MCParticleType'])
len(data_train)

# Eval generation

In [ ]:
index_sel = numpy.array(numbers_particles_df.index) % 2 == 1
numbers_particles_eval = numbers_particles_df[index_sel]

get_sample(numbers_particles_eval, 1000000, selection, dir_path + '/data_eval.csv', dir_path + '/get_sample_eval.log')

In [ ]:
data_eval = pandas.read_csv(dir_path + '/data_eval.csv', index_col=0, usecols=['MCParticleType'])
len(data_eval)